# 4  Pandas: работа с источниками данных

## 4.1 Что такое формат CSV и как его приручить?

In [12]:
import pandas as pd
import numpy as np

In [14]:
from pandas.core.computation.check import NUMEXPR_INSTALLED

**.csv - comma separated values**

Способы открытия файла:

In [17]:
df = pd.read_csv('E:\\Education\\tinkoff_course\\python\\bonus.csv', sep = ';')

In [18]:
df = pd.read_csv('E:/Education/tinkoff_course/python/bonus.csv', sep = ';')

In [19]:
df = pd.read_csv(r'E:\Education\tinkoff_course\python\bonus.csv', sep = ';')

**pd.read_csv(**

    filepath_or_buffer - путь,
    sep=',', - разделитель
    delimiter=None, - ??
    header='infer', - Заголовок, если нет, то указываем None
    names=None, - список из заголовков
    index_col=' ' - Выбираем индексы из колонки
    и.т.
)

**Еще несколько полезных параметров функции read_csv:**

`dtype` - тип данных в колонках

`usecols` - ограничить импорт выбранными колонками

`nrows` - количество строк для импорта

`encoding` - кодовая страница читаемого файла

`decimal` - символ десятичной точки

`thousonds` - символ разделителя тысяч

`skipinitialspace` - пропускать пробелы после разделителя

### Задача

⭐ **Определите название станции, на которой было наибольшее количество входов за 4 квартал 2021 года**

In [21]:
#df = pd.read_csv(r'E:\Education\data\Пассажиропоток_МосМетро_2.csv', sep = ';', header=1)

In [22]:
df = pd.read_csv(r'E:\Education\data\Пассажиропоток_МосМетро_2.csv', sep=';', skiprows=[1])

In [23]:
df[(df['Year'] == 2021) & (df['Quarter'] == 'IV квартал')]\
    .groupby(['NameOfStation'], as_index = False)\
    .agg({'IncomingPassengers':'sum'})\
    .sort_values(by= 'IncomingPassengers', ascending=False)\
    .head(1)

,NameOfStation,IncomingPassengers
74,Комсомольская,10105141


In [24]:
df[(df['Year'] == 2021) & (df['Quarter'] == 'IV квартал')][['NameOfStation','IncomingPassengers']]\
    .sort_values(by = 'IncomingPassengers', ascending=False)\
    .head(1)

,NameOfStation,IncomingPassengers
956,Комсомольская,7660212


In [25]:
df.loc[(df['Year'] == 2021) & (df['Quarter'] == 'IV квартал'), ['NameOfStation', 'IncomingPassengers']]\
    .sort_values(by='IncomingPassengers', ascending=False)\
    .head(1)

,NameOfStation,IncomingPassengers
956,Комсомольская,7660212


## 4.2 Поподробнее про обработку пропусков

`skiprows` = range(1, 4) - диапозон строк, которых нужно пропустить

Если вы встретите CSV файл, где разделителем будет TAB (\t), то такой файл можно прочитать в датафрейм несколькими способами:

    1) `read_csv('example.csv', sep='\t')`  т.к. по умолчанию сепаратор у `read_csv` запятая, поэтому мы задаем другой разделитель

    2) воспользоваться функцией `read_table`. Это тоже что и read_csv, но разделитель по умолчанию `'\t'`

Если NA реальное значение (а не пропуск), тогда можно поступить несколькими способами:

    1) Если ты уверен, что у тебя в данных нет пропусков, то можно задать na_filter=False и тогда пандас на этапе чтения не будет проверять значения и конвертировать в пропуск. 

    2) Отключить стандартный список пропусков через параметр keep_default_na = False, а затем задать свой список пропусков через параметр na_values=[‘#N/A’, ‘#N/A N/A’,  .....]

skiprows = 1 удаляет одну строку сверху, а skiprows=[1] удаляет вторую строку (которая под номером 1)

In [27]:
#nrows - чтение только первых n строк, не считая заголовка
pd.read_csv('E:\\Education\\tinkoff_course\\python\\bonus.csv', sep = ';', nrows = 5 )

,person_id,bonus
0,905,85059.638382
1,836,7703.346074
2,287,3120.269742
3,548,5347.987142
4,575,137257.490614


### Задача

⭐**Определите сколько всего было входов по всем станциям за всё время (сумма по столбцу IncomingPassengers)**

In [29]:
df2 = pd.read_csv(r'E:\Education\data\Пассажиропоток_МосМетро_3.csv', sep='|', header = 4, skiprows = [5, 6, 1380])

In [30]:
df2['IncomingPassengers'].fillna(0).sum()

2514195123.0

⭐**В столбце IncomingPassengers присутствуют пропущенные значения: 0, NULL, None, не указано. В ответе укажите число строк в которых пропущено значение в столбце IncomingPassengers**

In [31]:
df3 = pd.read_csv(r'E:\Education\data\Пассажиропоток_МосМетро_4.csv', sep = '|', 
                  skiprows = [1], na_values = {'IncomingPassengers' : [0, 'NULL', 'None', 'не указано']})

In [32]:
#1
df3.IncomingPassengers.isnull().value_counts()

False    1335
True       37
Name: IncomingPassengers, dtype: int64

In [33]:
#2
df3['IncomingPassengers'].isna().sum()

37

## 4.3 Учимся читать большие файлы кусочками

In [35]:
df

,NameOfStation,Line,Year,Quarter,IncomingPassengers,OutgoingPassengers,global_id,NameOfStation_en,Line_en,Year_en,Quarter_en,IncomingPassengers_en,OutgoingPassengers_en,Unnamed: 13
0,Митино,Арбатско-Покровская линия,2021,I квартал,1913498,1829031,1138975996,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Волоколамская,Арбатско-Покровская линия,2021,I квартал,1236714,1222309,1138975997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Строгино,Арбатско-Покровская линия,2021,I квартал,1938816,1903731,1138975999,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Крылатское,Арбатско-Покровская линия,2021,I квартал,1849616,1818208,1138976000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Площадь Революции,Арбатско-Покровская линия,2021,I квартал,2324687,2319343,1138976008,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,Китай-город,Таганско-Краснопресненская линия,2022,I квартал,0,0,2363487484,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1368,Киевская,Филёвская линия,2022,I квартал,0,0,2363487549,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1369,Улица Милашенкова,Московская монорельсовая транспортная система,2021,IV квартал,17991,17295,2363643842,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1370,Кунцевская,Филёвская линия,2022,I квартал,543507,529972,2363644363,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df4 = df[df['NameOfStation'] == 'Митино']

In [37]:
df4.to_csv("output_df4.csv", 
           columns = ['NameOfStation', 'Line', 'Year', 'Quarter', 'IncomingPassengers', 'OutgoingPassengers'],
          index = False,
          header = False,
          na_rep = 'NaN')

### Задача

⭐ **Вы оказались в аналитическом отделе некоторой социальной сети, которая объединяет людей занимающихся благотворительностью. Для вас подготовили выгрузку пользователей в формате csv. Загрузите файл в датафрейм и оставьте только пользователей женского пола. Сохраните датафрейм в формате csv без индекса и в кодировке utf8. Оставьте только две колонки: username и mail Разделитель значений: точка с запятой. Порядок записей не меняйте.**

In [40]:
df5 = pd.read_csv('e:users.csv', sep = ';')

In [41]:
df6 = df5[df5['sex'] == 'F'][['username', 'mail']]

In [42]:
df6.to_csv('women_mail.csv', index = False, sep = ';')

⭐ **Прочитайте первые 50 записей выгрузки и определите количество мужчин.**

In [43]:
df7 = pd.read_csv('e:users.csv', sep = ';', nrows=50)

In [44]:
df7[df7['sex'] == 'M']['sex'].value_counts()

M    27
Name: sex, dtype: int64

In [45]:
df7['sex'].value_counts()

M    27
F    23
Name: sex, dtype: int64

⭐ **Скачайте выгрузку и организуйте её чтение чанками. Размер каждого чанка установите в 30 строк. Для пятого чанка с данными определите сколько пользователей с группой крови A+**

In [46]:
df_ch = pd.read_csv('e:users.csv', sep = ';', chunksize=30)

In [47]:
#1
count = 0
for df in df_ch:
    count += 1
    if count == 5:
        print(df[df['blood_group'] == 'A+']['blood_group'].value_counts())
        break

A+    4
Name: blood_group, dtype: int64


In [ ]:
#2
for i in range(5):
    df = next(df_ch)
print(len(df[df.blood_group == 'A+']))

## 4.4 Кто такой JSON и как с ним подружиться?

`JSON`- JavaScript Object Notation

In [50]:
df8 = pd.read_json("e:test1.json")

In [51]:
df8

,surname,name,age,city
0,Ivanov,Ivan,18,Kazan
1,Petrov,Petr,27,Ufa


In [52]:
df6.head(2)

,username,mail
0,lambertemma,vsmith@yahoo.com
1,greid,graceholloway@yahoo.com


In [53]:
#orient =  по записями
df6.to_json('output.json', orient = 'records')

http://jsonviewer.stack.hu/

https://jsonformatter.org/

### Задачи

⭐ *Загрузите данные в датафрейм и определите самый дешевый тариф. Стоимость указана в столбце `TicketCost`, а наименование тарифа `NameOfTariff`*

In [54]:
df9 = pd.read_json('e:data-399-2022-07-01.json', encoding = 'Windows-1251')

In [55]:
df9.head(2)

,ID,NameOfTariff,TicketZone,global_id,NumberOfZone,TariffDistance,TypeOfTransport,NameOfCarrier,TicketCost,TicketValidity
0,200,«Единый» не более 2 поездок,"[{'TicketZone': 'Зона А'}, {'TicketZone': 'Зон...",19368709,-,-,"Метрополитен, монорельс, автобус, троллейбус, ...",[{'NameOfCarrier': 'ГУП «Московский метрополит...,114.0,"5 дней, включая день продажи"
1,700,«Единый» не более 60 поездок,"[{'TicketZone': 'Зона А'}, {'TicketZone': 'Зон...",19368716,-,-,"Метрополитен, монорельс, автобус, троллейбус, ...",[{'NameOfCarrier': 'ГУП «Московский метрополит...,1970.0,"45 дней, включая день продажи"


In [56]:
#1
df9['NameOfTariff'].iloc[df9['TicketCost'].idxmin()]

'«Разовый» льготный'

In [57]:
#
df9.loc[df9['TicketCost'] == df9.TicketCost.min()]['NameOfTariff']

17    «Разовый» льготный
Name: NameOfTariff, dtype: object

In [58]:
#3
df9.NameOfTariff[df9.TicketCost == df9.TicketCost.min()]

17    «Разовый» льготный
Name: NameOfTariff, dtype: object

⭐*Загрузите ее в датафрейм, оставьте только столбцы username, name, sex и сохраните в формате json (кодировка utf8). Сохранение выполните таким образом чтобы ключами были названия колонок, а значениями вложенные словари, которые будут хранить лейбл строки и само значение:*

In [59]:
df10 = pd.read_csv('e:users (1).csv', sep = ';', usecols=['username', 'name', 'sex'])

In [60]:
df10.head(2)

,username,name,sex
0,lambertemma,Stacy Reed,F
1,greid,Carol Farley,F


In [61]:
df10.to_json('users_step10.json')

## 4.5 Знакомимся с форматами HTML, XML, PICKLE, HDF5

In [ ]:
!pip install beautifulsoup4

In [63]:
_lst = pd.read_html('e:page_bank.html')

In [64]:
len(_lst)

1

In [65]:
df11 = _lst[0]

In [66]:
df11.drop(0, inplace = True)

In [67]:
df11.head(2)

,Государственный регистрационный номер выпуска,Эмитент,Цена (в процентах от номинала) одной ценной бумаги,"Cтоимость одной ценной бумаги, определенная в порядке, установленном Банком России (руб.)","Поправочный коэффициент, установленный Банком России",ISIN,Дата погашения*
1,12840061V,МИНФИН,1122135,"14 042 891,1294",098,XS0767473852,03.04.2042
2,12840068V,МИНФИН,1040408,"13 020 123,49152",098,XS0971721450,15.09.2023


In [68]:
df11.to_csv('output_page.cvs', index=False)

### Задачи

⭐ **Ваш друг Илья работает аналитиком в компании, которая занимается установкой и обслуживанием платных стоянок (стоек) для парковки велосипедов. Сегодня на работе ему поручили посчитать вместимость всех велосипедных стоянок в районе Тропарёво-Никулино. Говорит, что компания планирует устанавливать новые платные стоянки для велосипедов в этом районе. Проблема заключается в том, что данные лежат в формате HDF5 и Илья никогда раньше не работал с ним. Он попросил вашей помощи и сказал, что район записан в столбце District, а вместительность парковки в столбце Capacity**

**Посчитайте суммарную вместительность велосипедных парковок в районе Тропарёво-Никулино.**

In [69]:
 pd.HDFStore('e:data_store2.h5').keys()

['/parking_table', '/ser_district_value_counts']

In [70]:
df12 = pd.read_hdf('e:data_store2.h5', 'parking_table', mode = 'a')

In [71]:
df12[df12['District'] == 'район Тропарёво-Никулино']['Capacity'].sum()

26

⭐ **Загрузите данные в датафрейм и определите суммарный общий прирост постоянного населения за период с 2014 по 2020 (включительно) для субъекта Камчатский край**

**Т.е. мы ходим понять, что произошло с приростом за этот период: был ли он положительным или отрицательным и в каком количестве.**

In [72]:
data = pd.read_html('e:generation.html')
df13 = data[0]

In [73]:
#1
df13[df13['Unnamed: 0'] == 'Камчатский край']\
    .iloc[:, 3:10]\
    .T\
    .astype('int')\
    .sum()

94   -8197
dtype: int64

In [74]:
#2
data = pd.read_html('e:generation.html', 
                    skiprows = [0,1,3], index_col = 0) 
df14 = data[0] 
df14.drop(df14.columns[[0,1,9,10,11,12,13,14,15,16]], axis = 1, inplace = True) 
print(df14.loc['Камчатский край', :].sum())

-8197.0


⭐ **Загрузите данные в датафрейм и определите в какой из областей произошел наибольший отрицательный прирост постоянного населения за 2020 год (сравниваем абсолютные значения)**

**вариант из списка**

In [76]:
data = pd.read_html('e:generation.html', 
                    skiprows = [0,1,3], index_col = 0) 
df15 = data[0] 
df15.drop(df15.columns[[0,1,9,10,11,12,13,14,15,16]], axis = 1, inplace = True) 

In [77]:
df15.loc[['Свердловская область', 
          'Магаданская область',
          'Сахалинская область', 
          'Калужская область',
          'Ярославская область', 
          'Кировская область' ], '2020 г.'].idxmin()

'Свердловская область'

⭐ **Вы продолжаете работу в аналитическом отделе некоторой социальной сети, которая объединяет людей занимающихся благотворительностью. Коллеги аналитики для вас подготовили выгрузку пользователей в формате XML. Загрузите файл XML в датафрейм и посчитайте сколько пользователей женского пола с группой крови B+ в выгрузке.**

In [78]:
df16 = pd.read_xml('e:users.xml')

In [79]:
#1
df16[(df16['sex'] == 'F') & (df16['blood_group'] == 'B+')].shape[0]

5

In [80]:
#2
df16.loc[(df16['sex']=='F')&(df16['blood_group']=='B+'), 'sex'].value_counts()

F    5
Name: sex, dtype: int64

## 4.6 Старый добрый Excel и немного про базы данных

In [ ]:
# просмотреть все имена листов
xl.sheet_names

In [ ]:
pd.read_excel()

In [82]:
import sqlite3

In [83]:
con = sqlite3.connect('e:local_db.db')

In [84]:
cursor = con.execute("select * from stations")

In [85]:
cursor.description

(('ID', None, None, None, None, None, None),
 ('Name', None, None, None, None, None, None),
 ('StationCapacity', None, None, None, None, None, None),
 ('BikeParkingSlotsAmount', None, None, None, None, None, None),
 ('global_id', None, None, None, None, None, None),
 ('Longitude_WGS84', None, None, None, None, None, None),
 ('Latitude_WGS84', None, None, None, None, None, None),
 ('AdmArea', None, None, None, None, None, None),
 ('District', None, None, None, None, None, None),
 ('Location', None, None, None, None, None, None),
 ('ClarificationOfLocation', None, None, None, None, None, None),
 ('DepartmentalAffiliation', None, None, None, None, None, None),
 ('OperOrgName', None, None, None, None, None, None),
 ('OperOrgWebsite', None, None, None, None, None, None),
 ('Photo', None, None, None, None, None, None),
 ('ID_en', None, None, None, None, None, None),
 ('Name_en', None, None, None, None, None, None),
 ('StationCapacity_en', None, None, None, None, None, None),
 ('BikeParkingSl

In [86]:
row = cursor.fetchall()

In [87]:
df17 = pd.DataFrame(row, columns=[x[0] for x in cursor.description])

In [88]:
df17

,ID,Name,StationCapacity,BikeParkingSlotsAmount,global_id,Longitude_WGS84,Latitude_WGS84,AdmArea,District,Location,...,AdmArea_en,District_en,Location_en,ClarificationOfLocation_en,DepartmentalAffiliation_en,OperOrgName_en,OperOrgWebsite_en,Photo_en,geodata_center,geoarea
0,2,Пункт проката велосипедов № 2,14,None,2757552,37.5997037,55.7468663,Центральный административный округ,район Хамовники,"Гоголевский б-р (напротив д.17, стр.1)",...,None,None,None,None,None,None,None,None,None,None
1,3,Пункт проката велосипедов № 3,12,None,2757553,37.6005927778,55.7509180556,Центральный административный округ,район Арбат,Гоголевский б-р (напротив д.33),...,None,None,None,None,None,None,None,None,None,None
2,4,Пункт проката велосипедов № 5,12,None,2757554,37.5999825,55.7553104,Центральный административный округ,Пресненский район,"Никитский б-р (напротив д.11/12, стр.3)",...,None,None,None,None,None,None,None,None,None,None
3,5,Пункт проката велосипедов № 6,15,None,2757555,37.598667,55.757163,Центральный административный округ,Пресненский район,Никитский б-р (напротив д.23/14/9),...,None,None,None,None,None,None,None,None,None,None
4,7,Пункт проката велосипедов № 8,14,None,2757557,37.601473,55.761072,Центральный административный округ,Пресненский район,"Тверской б-р (напротив д.17, стр.1)",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722,1003,Пункт проката велосипедов № 966,15,None,2372463065,37.347834,55.690885,Западный административный округ,Можайский район,"Сколково, Большой бульвар, д.42, стр.1",...,None,None,None,None,None,None,None,None,None,None
723,1004,Пункт проката велосипедов № 967,15,None,2372463169,37.359745,55.697622,Западный административный округ,Можайский район,"Сколково, Большой бульвар, д.30, стр.1",...,None,None,None,None,None,None,None,None,None,None
724,1005,Пункт проката велосипедов № 968,15,None,2372463294,37.373916,55.706386,Западный административный округ,Можайский район,"Сколково, Большой бульвар, д.7",...,None,None,None,None,None,None,None,None,None,None
725,1006,Пункт проката велосипедов № 969,15,None,2372463404,37.350457,55.696231,Западный административный округ,Можайский район,"Сколково, Большой бульвар, д.47, к.1",...,None,None,None,None,None,None,None,None,None,None


### Задача

⭐ В базе данных `sqlite3` создана таблица `stations`, которая содержит информацию о местоположении станций велопроката, количестве мест на станции и слотов для парковки велосипедов. 

Выполните действия:

1) Выгрузите имя станции (столбец `Name`),  количество мест на станции (столбец `StationCapacity`), адресный ориентир (столбец `Location`). 

2) Выполните сортировку датафрейма по правилам

    сначала идут станции с наибольшим количеством мест
    
    если у нескольких станций одинаковое количество мест, то отсортируйте их в алфавитном порядке
    
3) Сохраните датафрейм в формате csv: без индекса, разделитель точка с запятой, кодировка utf8

In [89]:
import sqlalchemy as sql

In [90]:
con2 = sql.create_engine("sqlite:///e:local_db.db")

In [91]:
df18 = pd.read_sql("select Name, StationCapacity, Location from stations", con2)

In [92]:
df18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 727 entries, 0 to 726
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Name             727 non-null    object
 1   StationCapacity  727 non-null    object
 2   Location         727 non-null    object
dtypes: object(3)
memory usage: 17.2+ KB


In [93]:
df18['StationCapacity'] = df18['StationCapacity'].astype('int')

In [94]:
df18 = df18.sort_values(by=['StationCapacity', 'Name'], ascending=[False, True])

In [95]:
df18.to_csv('stations_task.csv', index=False, sep = ';', encoding='utf8')

In [ ]:
pd.to_sql(if_exist = 'replace')

⭐ Ваш старый друг Андрей купил электрокар Tesla Model X, а теперь подумывает о покупке квартиры в другом районе Москвы. Андрей устал, что в его районе почти всегда заняты зарядные станции и подзарядка машины превратилась в проблему, поэтому ему важно, чтобы на районе было много зарядных станций. 

Порекомендуйте Андрею ТОП-3 района по количеству зарядных станций. Андрей уже подготовил для вас выгрузку в Excel - данные на двух листах. 

In [111]:
xl = pd.ExcelFile('e:Зарядные_станции_для_электромобилей.xlsx')

In [112]:
xl.sheet_names

['0', '1']

In [113]:
df19 = pd.read_excel(xl, sheet_name='0', header=1)

In [118]:
df20 = pd.read_excel(xl, sheet_name='1', header=10)

In [121]:
df21 = pd.concat([df19, df20], ignore_index=True)

In [127]:
df21.head(2)

,Код,Наименование,Административный округ,Район,Адрес,Долгота в WGS-84,Широта в WGS-84,global_id,ID_en,Name_en,AdmArea_en,District_en,Address_en,Longitude_WGS84_en,Latitude_WGS84_en,geodata_center,geoarea
0,25586,"ЭЗС Колпачный переулок дом 3, строение 2",Центральный административный округ,Басманный район,"город Москва, Колпачный переулок, дом 3, строе...",37.643166,55.758317,1036018683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25588,ЭЗС улица Макаренко дом 1/19,Центральный административный округ,Басманный район,"город Москва, улица Макаренко, дом 1/19",37.646220,55.761024,1036018684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
df21['Район'].value_counts().nlargest(3)

Пресненский район    11
Мещанский район      10
Басманный район       9
Name: Район, dtype: int64